# 0. Setup Paths

In [1]:
import os
# import wget
# import object_detection

## Note on setup
`SETUP_ENVIRONMENT` controls whether tensorflow and related packages need to be downloaded, configured and installed. This is a one-time operation for a new user.
All in all, this script considers a clean environment with python=3.6, and protobuf-compiler installed (`sudo apt-get install protobuf-compiler`)

In [2]:
SETUP_ENVIRONMENT = True
CUSTOM_MODEL_NAME = 'my_centernet_hg104_1024_8'
PRETRAINED_MODEL_NAME = 'centernet_hg104_1024x1024_coco17_tpu-32'
PRETRAINED_MODEL_URL = 'http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

In [3]:
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations',CUSTOM_MODEL_NAME),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'IMAGE_PATH_TRAIN': os.path.join('Tensorflow', 'workspace','images','train'),
    'IMAGE_PATH_TEST': os.path.join('Tensorflow', 'workspace','images','test'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'), 
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'), 
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

In [4]:
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME), 
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [5]:
# creating folders
for path in paths.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

In [6]:
print('Using the following models and annotation paths:')
print(paths['CHECKPOINT_PATH'])
print(paths['ANNOTATION_PATH'])

Using the following models and annotation paths:
Tensorflow/workspace/models/my_centernet_hg104_1024_8
Tensorflow/workspace/annotations/my_centernet_hg104_1024_8


# Download TF models from Tensorflow Model Zoo and install TFOD

In [7]:
if os.name=='nt':
    !pip install wget
    import wget

In [8]:
print(f"Searching for path: {paths['APIMODEL_PATH']}/research/object_detection...")
if not os.path.exists(os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection')):
    print('Path not found - cloning from repository...')
    !git clone https://github.com/tensorflow/models {paths['APIMODEL_PATH']}
else:
    print('Path found... Skipping')

Searching for path: Tensorflow/models/research/object_detection...
Path found... Skipping


In [9]:
# Install Tensorflow Object Detection 
if SETUP_ENVIRONMENT:
    if os.name=='posix':  
        !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install . 
        
    if os.name=='nt':
        url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
        wget.download(url)
        !move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
        !cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
        os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))   
        !cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
        !cd Tensorflow/models/research/slim && pip install -e . 

Processing /home/ivo/Development/tensorflow-object-detection/Tensorflow/models/research
  Preparing metadata (setup.py) ... done
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=21902073 sha256=800b3758773823e801602fceb4a16a285714d3a9f8cc0f11b67fd5434662f408
  Stored in directory: /tmp/pip-ephem-wheel-cache-frw7hvi2/wheels/a1/d2/27/2ad15538537efda79e9e94213bdf505578bbb5da2d5ba29693
Successfully built object-detection
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
You should consider upgrading via the '/home/ivo/anaconda3/envs/tfod/bin/python -m pip install --upgrade pip' command.


In [10]:
if SETUP_ENVIRONMENT:
    VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
    # Verify Installation
    !python {VERIFICATION_SCRIPT}

2023-03-24 12:02:49.977387: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 12:02:50.009784: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 12:02:50.010091: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.6.13: /home/ivo/anaconda3/envs/tfod/bin/python
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2023-03-24 12:02:50.018018: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructi

In [11]:
if SETUP_ENVIRONMENT:
    !pip install tensorflow --upgrade

You should consider upgrading via the '/home/ivo/anaconda3/envs/tfod/bin/python -m pip install --upgrade pip' command.


# 1. Load pre-trained Model

In [12]:
#!pip list
!nvidia-smi

Fri Mar 24 12:03:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.108.03   Driver Version: 510.108.03   CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   38C    P0    55W / 200W |    102MiB /  8192MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
!who

mattia   :1           2023-01-16 16:17 (:1)
ivo      pts/9        2023-03-24 08:46 (82.170.226.202)
ivo      pts/15       2023-03-24 10:22 (82.170.226.202)


## Downloading the model
On tf Model Zoo Website (https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)
- right click on model to get url
- copy name part of url to use as name

In [14]:
# download model from tf model zoo
if os.name =='posix':
    !wget {PRETRAINED_MODEL_URL}
    !mv {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}
if os.name == 'nt':
    wget.download(PRETRAINED_MODEL_URL)
    !move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
    !cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

--2023-03-24 12:03:16--  http://download.tensorflow.org/models/object_detection/tf2/20200713/centernet_hg104_1024x1024_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.179.208, 2a00:1450:400e:803::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.179.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1424430731 (1,3G) [application/x-tar]
Saving to: ‘centernet_hg104_1024x1024_coco17_tpu-32.tar.gz’

centernet_hg104_102 100%[===================>]   1,33G  33,3MB/s    in 19s     

2023-03-24 12:03:35 (72,5 MB/s) - ‘centernet_hg104_1024x1024_coco17_tpu-32.tar.gz’ saved [1424430731/1424430731]

centernet_hg104_1024x1024_coco17_tpu-32/
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/checkpoint
centernet_hg104_1024x1024_coco17_tpu-32/checkpoint/ckpt-0.index
centernet_h

# 2. Create Label Map

In [15]:
labels = [{'name':'Bubble', 'id':1}]

# write into label map file
with open(files['LABELMAP'], 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

# 3. Create TF records
- binary storage format
- convert annotations and images into format for training
- storing data as sequence of binary strings has impact on training time
- (less space on disk, copy faster, read efficiently)

In [16]:
# create TFRecord file
# --> train and test record file within annotations path
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_8/train.record
Successfully created the TFRecord file: Tensorflow/workspace/annotations/my_centernet_hg104_1024_8/test.record


# 4. Copy Model Config to Training Folder
- pipeline.config defines architecture of model
- contains e.g. prebuilt data augmentation
- see pipeline file in pre-trained model folder --> preset configurations (e.g. data augmentation etc.)

In [17]:
# copy pipeline.config into my_model folder
if os.name =='posix':
    !cp {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}
if os.name == 'nt':
    !copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# 5. Update Config For Transfer Learning
config file: change parameter of the model

In [18]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [19]:
config = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])

In [20]:
config

{'model': center_net {
   num_classes: 90
   feature_extractor {
     type: "hourglass_104"
     channel_means: 104.01361846923828
     channel_means: 114.03422546386719
     channel_means: 119.91659545898438
     channel_stds: 73.60276794433594
     channel_stds: 69.89082336425781
     channel_stds: 70.91507720947266
     bgr_ordering: true
   }
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 1024
       max_dimension: 1024
       pad_to_max_dimension: true
     }
   }
   object_detection_task {
     task_loss_weight: 1.0
     offset_loss_weight: 1.0
     scale_loss_weight: 0.10000000149011612
     localization_loss {
       l1_localization_loss {
       }
     }
   }
   object_center_params {
     object_center_loss_weight: 1.0
     classification_loss {
       penalty_reduced_logistic_focal_loss {
         alpha: 2.0
         beta: 4.0
       }
     }
     min_box_overlap_iou: 0.699999988079071
     max_box_predictions: 100
   }
 }, 'train_config': batch_si

In [21]:
#ckeys = config.keys()
#print(type(ckeys))
#print(list(ckeys)[0])

In [22]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "r") as f:                                                                                                                                                                                                                     
    proto_str = f.read()                                                                                                                                                                                                                                          
    text_format.Merge(proto_str, pipeline_config)  

MODIFY FIRST LINE!
- pipeline_config.model.###
- insert model name (shown in first line of config file)

In [23]:
# modify pipeline.config for personal use case (num classes, set paths etc.)
pipeline_config.model.center_net.num_classes = len(labels)
pipeline_config.train_config.batch_size = 1
pipeline_config.train_config.fine_tune_checkpoint = os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'checkpoint', 'ckpt-0')
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path= files['LABELMAP']
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'train.record')]
pipeline_config.eval_input_reader[0].label_map_path = files['LABELMAP']
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [os.path.join(paths['ANNOTATION_PATH'], 'test.record')]

In [24]:
print([os.path.join(paths['ANNOTATION_PATH'], 'test.record')])

['Tensorflow/workspace/annotations/my_centernet_hg104_1024_8/test.record']


In [25]:
config_text = text_format.MessageToString(pipeline_config)                                                                                                                                                                                                        
with tf.io.gfile.GFile(files['PIPELINE_CONFIG'], "wb") as f:                                                                                                                                                                                                                     
    f.write(config_text)   

# 6. Train the model
Before Training, my_model directory must be empty!

In [26]:
TRAINING_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')

In [27]:
# test if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# check GPU use
# watch -n0.1 nvidia-smi

Num GPUs Available:  1


In [28]:
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps=20000".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'])

In [29]:
# to see training status, copy command to prompt and run from there
# (from tfodCourse directory and with activated environment)
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_8 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_8/pipeline.config --num_train_steps=20000


- runs model_main_tf2.py
- arguments passed:
    - model directory
    - directory of pipeline.config
    - number of training steps

In [30]:
#!{command}

# 7. Evaluate the Model
Calculate performance metrics

In [31]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(TRAINING_SCRIPT, paths['CHECKPOINT_PATH'],files['PIPELINE_CONFIG'], paths['CHECKPOINT_PATH'])

In [32]:
# command can be run from prompt to see progress
print(command)

python Tensorflow/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_8 --pipeline_config_path=Tensorflow/workspace/models/my_centernet_hg104_1024_8/pipeline.config --checkpoint_dir=Tensorflow/workspace/models/my_centernet_hg104_1024_8


In [33]:
#!{command}

# See Evaluation graphically with Tensorboard
- Eval folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\eval --> AV, AR
- Train folder: \TFODCourse\Tensorflow\workspace\models\my_ssd_mobnet\train --> losscurve

in prompt: go to directory

run command: tensorboard --logdir=.
    
open localhost url

## Ideas to improve performance
- more imgs of low performing classes to training dataset
- train longer
- change architecture